In [16]:
using JuMP
using Gurobi

In [17]:
# Load demand and resource structs
include(joinpath(@__DIR__, "..", "src", "core", "demand.jl"))
include(joinpath(@__DIR__, "..", "src", "core", "resource.jl"))

loadresources (generic function with 1 method)

In [18]:
# Find the example data
# Search parent directories till we find "Example_MESS"
function find_dir_upwards(dirname::String)
    root_dir = ""
    current_dir = @__DIR__
    while !(dirname in readdir(current_dir))
        current_dir = joinpath(current_dir, "..")
    end
    return joinpath(current_dir, dirname)
end

data_dir = find_dir_upwards("Example_MESS")

"/Users/rmacd/Documents/GitHub/Macro/prototypes/structs_demo_1/tests/../../../Example_MESS"

In [19]:
# Load example data
resources = loadresources(joinpath(data_dir, "resource_example.csv"))
demands = loadelecdemands(joinpath(data_dir, "demand_example.csv"))

1-element Vector{Demand}:
 Demand("zone_1", [8936.480641, 8437.556316, 8040.661505, 7712.591491, 7571.235488, 7481.914537, 7059.699734, 7137.242165, 7345.382375, 7893.836139  …  9961.213686, 9963.886339, 10697.47855, 10475.05421, 9896.54686, 9641.545275, 9454.831263, 9884.375519, 10002.72976, 9972.969566], "electricity", "components")

# Build model

In [20]:
M = JuMP.Model(Gurobi.Optimizer)

T = length(demands[1].demand)
R = length(resources)

JuMP.@variable(M, vCap[1:length(resources)] >= 0)
JuMP.@variable(M, vPow[1:T, 1:R] >= 0)

JuMP.@constraint(M, cPowerBalance[t=1:T], sum(vPow[t, r] for r in 1:R) == demands[1].demand[t])

JuMP.@constraint(M, cCapacity[t=1:T, r=1:R], vCap[r] >= vPow[t, r])

@expression(M, eFixCost, sum(vCap[r] * resources[r].fixedcost for r in 1:R))
@expression(M, eVarCost, sum(sum(vPow[t, r] for t=1:T) * resources[r].variablecost for r in 1:R))

@objective(M, Min, eFixCost + eVarCost)

optimize!(M)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-11
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 35040 rows, 26283 columns and 78840 nonzeros
JuMP.Model fingerprint: 0xd583adde
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+03, 1e+04]
Presolve time: 0.02s
Presolved: 35040 rows, 26283 columns, 78840 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 3
 AA' NZ     : 5.256e+04
 Factor NZ  : 1.927e+05 (roughly 26 MB of memory)
 Factor Ops : 1.104e+06 (less than 1 second per iteration)
 Threads    : 6

Barrier performed 0 iterations in 0.04 seconds (0.04 work units)
Barrier solve interrupted - model solved by